# Binary Numbers

Lexer, parser

| Syntactic rules| Semantic Rules |
|:-:|:-:|
| sec -> sec_1 `.` sec_2 | sec_1.v + sec_2.v / 2 ^ sec_2.lon |
| sec -> sec_1 digit | sec_1.v *  2 + dig.v, sec.lon = sec_1.lon + 1 |
| sec -> digit | sec.v = digit.v, sec.long = 1 |
| digit| digit.v = digit.lexval[01]

In [3]:
from sly import Lexer, Parser
import pandas as pd

In [2]:
class BinaryLexer(Lexer):
    literals = {'.'}
    tokens = {'DIGIT'}

    ignore = '\t'

    @_(r'[01]')
    def DIGIT(self, t):
        t.value = int(t.value)
        return t
    
    def error(self, t):
        print('<-'*10, "Illegal character ' {} ' ".format(t.value[0], '->' * 10))
        t.type = "Illegal"
        t.value = t.value[0]
        self.index +=1
        return t

In [6]:
data = pd.read_csv('assets\/binary.csv')
data

,binary
0,1010101.00
1,1010.00
2,1.00
3,0.00
4,1010101.11


In [7]:
lexer = BinaryLexer()
sentences = pd.read_csv('assets\/binary.csv').binary
pass_or_not = []
all_token_pass = True 

for index, sentence in enumerate(sentences):
    print('-' * 80, "{} Lexically Testing sentence: '{}'".format(index,
          sentence), '-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not
